In [8]:
import pandas as pd
from statsmodels.iolib.smpickle import load_pickle

# I/O
MURATES = "/data/ruderferlab/projects/biovu/cnv/cre/projects/CBS_FunctionalAnnotation/resources/data/gnomad/gnomad_v2.supplement-f10.murates-long.tsv"
NEUTRAL_MODEL = "results/manuscript/maps/maps-calibrated.pickle"
MATRIX = "/data/ruderferlab/projects/biovu/cnv/cre/projects/CBS_FunctionalAnnotation/results/manuscript/maps/matrix-subset.for_maps.tsv"

###
# Functions
###

def apply_status(df, unbound_field: str, bound_field: str):
    return np.where(
        df[unbound_field] == 1,
        "Unbound",
        np.where(
            df[bound_field] == 1,
            "Bound",
            np.where(
                df[unbound_field].isna() & df[bound_field].isna(), "NaN", "Ambiguous"
            ),
        ),
    )
    
def read_murates(filepath: str) -> pd.DataFrame:
    """Returns gnomAD murate table as pandas df"""
    return pd.read_csv(filepath, sep="\t", engine="c")


def calculate_maps(snvs: pd.DataFrame, factor: list, betas) -> pd.DataFrame:
    """D"""
    # Factors
    factor_one = factor[0]
    factor_two = factor[1]
    factor_three = factor[2]
    
    # Aggregate stats
    agg_matrix = (
        snvs.groupby(factor)
        .agg(
            {
                "singleton": "sum",
                "isvar": "sum",
                "methylation_level": "unique",
            }
        )
        .explode("methylation_level")
        .reset_index()
    )
    agg_matrix.rename(
        columns={
            "singleton": "singleton_count",
            "isvar": "context_nvar",
        },
        inplace=True,
    )

    # Update with expected singlton count
    agg_matrix["expected_singelton_count"] = (
        agg_matrix["mu_snp"] * betas.mu_snp + betas.const
    ) * agg_matrix["context_nvar"]
    
    #return agg_matrix

    # Summarize
    agg = agg_matrix.groupby([factor_one, factor_two, factor_three]).agg(
        {
            "singleton_count": "sum",
            "expected_singelton_count": "sum",
            "context_nvar": "sum",
        }
    )

    # Calc ps
    agg["ps"] = agg["singleton_count"] / agg["context_nvar"]

    # Calculate maps and sem
    agg["maps"] = (agg["singleton_count"] - agg["expected_singelton_count"]) / agg[
        "context_nvar"
    ]
    agg["sem"] = (agg["ps"] * (1 - agg["ps"]) / agg["context_nvar"]) ** 0.5

    # Flag factor
    agg.reset_index(inplace=True)
   
    # Return agg matrix
    return agg

## Main

In [11]:
# New matrix
matrix = pd.read_csv(MATRIX, sep="\t")

# Expected SP
model = load_pickle(NEUTRAL_MODEL)

# Extract betas for coeffs
betas = model.params

# Read murates
murate_df = read_murates(MURATES)

# Merge datasets
murate_keys = ["ref", "alt", "context"]
lobs = pd.merge(matrix, murate_df, on=murate_keys, how="left")

# Calc maps
maps =  calculate_maps(lobs, ["quantile_rdhswide", "dpwm_class", "pwm_stat_class", "mu_snp"], betas)

In [12]:
###
# Example output
###

maps.head()

,quantile_rdhswide,dpwm_class,pwm_stat_class,singleton_count,expected_singelton_count,context_nvar,ps,maps,sem
0,1,G,0,1491,1565.968454,3508,0.425029,-0.021371,0.008346
1,1,G,1,513,697.713137,1655,0.309970,-0.111609,0.011368
2,1,L,0,4054,4194.169210,9427,0.430041,-0.014869,0.005099
3,1,L,1,3919,4004.317706,8963,0.437242,-0.009519,0.005240
4,2,G,0,1038,1205.740930,2707,0.383450,-0.061966,0.009345


In [14]:
maps.to_csv("/data/ruderferlab/projects/biovu/cnv/cre/projects/CBS_FunctionalAnnotation/results/manuscript/maps/maps_scored.tsv", sep="\t", index=False)